**Code copied from this medium post - https://medium.com/geekculture/fine-tune-eleutherai-gpt-neo-to-generate-netflix-movie-descriptions-in-only-47-lines-of-code-40c9b4c32475** 

In [1]:
pip install git+https://github.com/huggingface/transformers@master

  Cloning https://github.com/huggingface/transformers (to revision master) to c:\users\asus\appdata\local\temp\pip-req-build-hg4wn6ux
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\ASUS\AppData\Local\Temp\pip-req-build-hg4wn6ux'
  Running command git checkout -q master
  error: pathspec 'master' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q master did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q master did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

In [2]:
!nvidia-smi

Tue Jul 11 19:09:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 L...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   56C    P8               15W /  N/A|      6MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
torch.manual_seed(42)

### Loading GPT2-Medium Model from 🤗 Model Hub 

In [5]:


tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium').cuda()
model.resize_token_embeddings(len(tokenizer))


Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Embedding(50259, 1024)

In [6]:
descriptions = pd.read_csv('../input/netflix-shows/netflix_titles.csv')['description']

In [7]:
max_length = max([len(tokenizer.encode(description)) for description in descriptions])

In [8]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [9]:
dataset = NetflixDataset(descriptions, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])


In [10]:
import gc
gc.collect()

20

In [11]:
torch.cuda.empty_cache()

In [12]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=1, logging_steps=100, save_steps=5000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')


In [13]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7926
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 7926


Step,Training Loss
100,5.472100
200,2.029000
300,1.915900
400,1.875800
500,1.901700
600,1.838800
700,1.903200
800,1.864000
900,1.837200
1000,1.901100


Saving model checkpoint to ./results/checkpoint-5000
Configuration saved in ./results/checkpoint-5000/config.json
Model weights saved in ./results/checkpoint-5000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7926, training_loss=1.8263790333719228, metrics={'train_runtime': 1330.6789, 'train_samples_per_second': 5.956, 'train_steps_per_second': 5.956, 'total_flos': 891356768944128.0, 'train_loss': 1.8263790333719228, 'epoch': 1.0})

### GPT Generated Description

In [14]:
generated = tokenizer("<|startoftext|> ", return_tensors="pt").input_ids.cuda()

In [15]:
sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [16]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0:  ̶Curious petals, quirky animals, magical beasts and stories around some of the world wide wonders from animal science, ecology and natural history series are told through illustrations.
1: ?????? (Seth) meets in Amsterdam with his bestest friends when Seth’s love of writing ends in failure. Will they find some happiness in the afterlife? Based on the original story. I Can’t Wait, I Can’te It, Is Myself On.
2:   What happens when one falls pregnant with her husband’s help?
3:  ̶ ̶Bollywood veteran director and activist, Satya Malhotra appears today at a meeting dedicated to women empowerment in Delhi to discuss "rape myths" she encountered while traveling.
4:  ” To take revenge on a corrupt, corrupt court and to keep his own wife and children safe” in modern society, corrupt prosecutor Kishore and an enforcer get very much in their relationship. That'll be his name!
5:  ㅋㅋThe funniest and scini-twinch comedians perform daily. But you decide exactly what the daily stand-up is until 

### Original Description (Random)

In [17]:
pd.options.display.max_colwidth = 1000
descriptions.sample(10)

4970                       Three buddies with big dreams go from underachieving slackers to badass warriors when their posh hotel is taken over by terrorists.
3362         In his first stand-up special, Arsenio Hall discusses getting older, the changing times and culture, social issues and even bothersome baby toes.
5494                                                  Music meets imagination in this inventive animated series about thinking outside the box and having fun.
1688                        Explore an array of unique competitions, from the quirky to the bizarre, and meet their passionate communities in this docuseries.
1349         From his days as a petty thief to becoming head of a drug-trafficking empire, this riveting series charts the life of the infamous Pablo Escobar.
4862        This anime adventure follows the battle between a saint of Athena and an avatar of Hades who's working on a painting that could destroy the world.
2676     A top Israeli agent comes out of reti